# Rent or own calculator for Alberta

## IMPORTANT DISCLAIMER

The calculations and results of this code are presented without any warranty. Don't make
a huge financial decision based off some code you found on the internet. While I've done
my best to make a good model, I've made a number of assumptions about the structure
of the problem. On top of that, the model itself requires entering a number of assumptions.
The model can only be said to be accurate if the calculations are correct, the structure
of the problem is correct, and the assumptions that the user enters to specify their exact
comparison are correct. I made this as a model to organize my thinking about the tradeoffs
of renting vs owning a home, and have made efforts to make it accurate, but I can't offer
any guarantees to its validity, and assume no responsibility for any financial decisions
anyone might make in whole or in part due to its results. Hope that covers my ass.

## How to run

With that said, here's the general idea of how to use this notebook. If you've never used a Jupyter notebook (this thing) before and want to read more about it, [this guide](https://www.dataquest.io/blog/jupyter-notebook-tutorial/) is a decent place to start. The TLDR for just running a simulation is that each of these grey blocks is a chunk of code. If you run them in order, updating the prompts that they provide as you go, you'll get to a "run simulation" button, which will output the results of simulating the financial outcome of renting or owning under the conditions that you specify. Each of the grey blocks (or cells) can be run by either selecting them and pressing ctrl+enter or clicking the play icon in the top bar. Don't forget to adjust the settings below each box before you run the simulation.

## First cell - setup

There are no prompts or updates here, this box just loads all the logic necessary to run the simulation. Select it and then press ctrl+enter to continue.

In [ ]:
%load_ext autoreload
%autoreload 2
import locale
import pandas as pd
import numpy as np
from rentorown.asset import annual_to_monthly_return, annual_to_monthly_stdev
from rentorown.rentorown import RentOrOwn
import ipywidgets as widgets
from IPython.display import display
locale.setlocale(locale.LC_ALL, '');
pd.options.display.max_rows = 999

## Rental characteristics

The only thing this model is concerned about regarding the rental property is the monthly rent. Don't include any costs that you'd incurr even if you owned (e.g utilities).

In [ ]:
style = {'description_width': 'initial'}
rent_box = widgets.IntText(
    value=1000,
    description="Monthly Rent:",
    disabled=False,
    style=style
)
display(rent_box)

## Ownership characteristics

There are a few more factors to consider on the owning side. The first is just the price of the house, which you can set after running the cell below.

### purchase price

In [ ]:
house_price_box = widgets.IntText(
    value=100_000,
    description="House purchase price: ",
    disabled=False,
    style=style
)
display(house_price_box)

### down payment
Set the down payment, either use the slider to select a percentage, or directly input a dollar amount in the next box

In [ ]:

down_payment_pct_box = widgets.FloatSlider(
    value=0.1,
    min=0,
    max=1.0,
    step=0.01,
    description="% down: ",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".0%",
    style=style
)
dp_dollar = widgets.Output()
display(down_payment_pct_box, dp_dollar)
def down_payment_event(btn_object):
    with dp_dollar:
        dp_dollar.clear_output()
        print(f"Down payment dollars: ${house_price_box.value * btn_object['new']:n}")


down_payment_pct_box.observe(down_payment_event, names="value")

In [ ]:
down_payment_box = widgets.IntText(
    value=20_000,
    description="Down payment in dollars",
    style=style
)
def dp_slider_change(change):
    down_payment_box.value = house_price_box.value * change.new

down_payment_pct_box.observe(dp_slider_change, names='value')
display(down_payment_box)

### amortization period

Set the amortization period for the mortgage

In [ ]:
mortgage_amortization_slider = widgets.IntSlider(
    value=25,
    min=1,
    max=30,
    step=1,
    description="Years of amortization: ",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
    style=style
)
display(mortgage_amortization_slider)

### mortgage rate

Set the APR rate for your mortgage. This is taken as fixed across your entire amortization period. So depending on your assumptions about future mortgage rates you may want to set it higher or lower than the currently available rates to account for adjustments over the life of your mortgage.

In [ ]:
mortgage_apr_slider = widgets.FloatSlider(
    value=0.03,
    min=0.0,
    max=0.3,
    step=0.001,
    description="Mortgage APR rate: ",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".1%"
)
display(mortgage_apr_slider)

### Investment assumptions

In the above sections we have defined prices, interest rates, and time horizons. The other important set of assumptions we need to make are around the returns you'll make on your investment, either appreciation in the case of purchasing a house, or investment returns from what would have gone to a down payment, and any net difference in monthly costs between renting and owning in the case of renting.

In the base version of this model I assume that both housing and investment returns are normally distributed. Later in the notebook I'll highlight where you can modify that assumption if you have a different distribution in mind. Because everything is calculated on a monthly basis it's necessary to enter the mean and standard deviation of expected returns in monthly terms. Since most people are more used to thinking about both of these factors in annual terms you can run the cells below to convert from annual to monthly results.

Note that you can re-run this cell by pressing ctrl+enter again if you want to compute multiple returns (for example to set different rates for housing and investment returns

In [ ]:
ann_return = float(input("Enter an annual rate of return in decimal form. e.g. 10% would be 0.1: "))
monthly_return = annual_to_monthly_return(ann_return)
print(f"In monthly terms that is equivalent to approximately {monthly_return:.4}")

In [ ]:
ann_stdev = float(input("Enter the standard deviation of an annual return in decimal form. e.g. 3% would be 0.03: "))
monthly_stdev = annual_to_monthly_stdev(ann_stdev)
print(f"In monthly terms that is equivalent to approximately {monthly_stdev:.4}")

#### Housing returns and standard deviation

Enter the (monthly) mean and standard deviation of returns you expect to housing. Recall that you can use the two cells above this to convert from annual to monthly

In [ ]:
housing_mean = widgets.FloatText(
    value=0.004,
    description="Monthly mean return to housing",
    disabled=False,
    style=style
)
display(housing_mean)

In [ ]:
housing_stdev = widgets.FloatText(
    value=0.0136,
    description="Monthly standard deviation to housing",
    disabled=False,
    style=style
)
display(housing_stdev)

In [ ]:
investment_mean = widgets.FloatText(
    value=0.00510,
    description="Monthly mean return to investment",
    disabled=False,
    style=style
)
display(investment_mean)

In [ ]:
investment_stdev = widgets.FloatText(
    value=0.0136,
    description="Monthly standard deviation to investment",
    disabled=False,
    style=style
)
display(investment_stdev)

### Final modifications and run the simulation

At this point all of the basic assumptions have been set. If you're happy with them you can run this cell (ctrl+enter) and click the "Run Simulation" button. It will then run 1,000 simulations of renting and owning under the scenario you have described above. Once those are complete it will plot a histogram of the distribution of the value of your asset (a house or investments) at the end of your mortgage amortization, as well as the median of those values over the investment horizon. Finally it will show a table of your mortgage payments over the amortization period if you were to buy a house.

There are a few remaining factors that can be tweaked here before you run the simulation. I haven't created sliders for them, but you can just modify the cell below directly if you'd like. I'll outline what those options are below.

The lines you'd want to modify to change the scenario start after the ```mtg = RentOrOwn``` line in the cell below. The first 5 lines below that are fully controlled by the sliders above. You can hard code numbers in there if you'd like, but in those cases it's probably easier to use the sliders. The next to lines refer to the assumptions about the distribution of housing and investment assets. As long as you're ok with the assumption of normally distributed returns you can just use the sliders above. If you want to use a different distribution you can modify those two lines. You can refer to the [numpy docs](https://numpy.org/doc/stable/reference/random/index.html) for details on specifying other distributions, a detailed guide is beyond the scope of this notebook.

Beyond those lines, there are a few smaller assumptions that you can modify. They are as follows:

* ```number_of_simulations``` How many times to simulate each scenario. The default is 1_000, which seems to be a decent tradeoff between getting a good sampling of possible outcomes with not making the simulation take too long. If you are running this on a nice beefy machine feel free to crank this up, and if you're running it somewhere less powerful and it's taking too long, try scaling it down
* ```additional_purchase_costs``` This is meant to cover things like legal fees, title insurance, home inspection, home appraisal, etc. If you leave it set to ```None``` it defaults to 2,300, which I estimated to be a reasonable amount when I wrote this code. If you know better for your particular situation and time feel free to update.
* ```additional_monthly_costs``` Any additional costs of ownership beyond a mortgage and property tax (e.g condo fees, or utilities that would be included in your rent)
* ```mortgage_payment_schedule``` how frequently you will make mortgage payments. The default is "monthly", but you can change it to any of "monthly", "bi_weekly" or "acc_bi_weekly" (which stands for accelerated bi-weekly.
* ```annual_inflation``` The annual rate at which rent and non-mortgage ownership costs will grow. I assume 2% by default based on the Bank of Canada's long run target.
* ```monthly_property_tax_rate``` this is the percentage of the initial home value that will be charged. If default will take a number that I pulled from the City of Edmonton calculator at time of writing. Note that this is escalated by inflation, not against the forecasted value of the home, since I make the assumption that cities are trying to pull in a fixed real level of earnings from residential real estate, and will move their rates up or down to adjust to changing real estate prices. Maybe that's not the right assumption, see what I mean about this being tricky?
* ```maintenance_cost``` This is the annual percentage of the starting value of the house that will have to go towards maintenance that you wouldn't incurr in the case of renting. It is also escalated by inflation. It defaults to 1%, but you may need to  modify it, especially in the case of an older house.


In [ ]:
calc_button = widgets.Button(description="Run Simulation")
out_widget = widgets.Output()
display(calc_button)
display(out_widget)
def calculate_rent_or_own(btn_object):
    with out_widget:
        out_widget.clear_output()
        mtg = RentOrOwn(
            monthly_rent=rent_box.value,
            house_price=house_price_box.value,
            down_payment=down_payment_box.value,
            mortgage_amortization_years=mortgage_amortization_slider.value,
            mortgage_apr=mortgage_apr_slider.value,
            housing_asset_dict={"dist": np.random.normal, "dist_args": {"loc": housing_mean.value, "scale": housing_stdev.value}},
            investment_asset_dict={"dist": np.random.normal, "dist_args": {"loc": investment_mean.value, "scale": investment_stdev.value}},
            number_of_simulations=1_000,
            additional_purchase_costs=None,
            additional_monthly_costs=0,
            mortgage_payment_schedule="monthly",
            mortgage_additional_payments=0,
            annual_inflation=0.02,
            monthly_property_tax_rate=None,
            maintenance_cost=0.01
        )
        mtg.histogram()
        mtg.median_returns_plot()
        display(mtg.mortgage_df)

calc_button.on_click(calculate_rent_or_own)
